In [ ]:
import bibliograph as bg

bibtex_fname = "bibliograph/test_data/bibtex_test_data_short.bib"
entry_syntax_fname = "bibliograph/resources/default_bibtex_syntax.csv"

tn = bg.slurp_bibtex(
    bibtex_fname,
    entry_syntax_fname,
    syntax_case_sensitive=False,
    space_char='|',
    na_string_values='!',
    na_node_type='missing'
)

tn.strings

In [10]:
import bibliograph as bg

tn = bg.slurp_shorthand(
    'bibliograph/test_data/manual_annotation.shnd',
    "bibliograph/resources/default_entry_syntax.csv",
    link_syntax_fname="bibliograph/resources/default_link_syntax.csv",
    syntax_case_sensitive=False,
    item_separator='__',
    default_entry_prefix='wrk',
    space_char='|',
    na_string_values='!',
    na_node_type='missing',
    skiprows=2,
    comment_char='#'
)

shorthand_text_node_type_id = tn.id_lookup(
    'node_types',
    'shorthand_text',
    column_label='node_type'
)

tn.strings.query('node_type_id == @shorthand_text_node_type_id')

,string,node_type_id,date_inserted,date_modified
60,This is stuff shorthand ignores when you use s...,0,2022-07-13 17:11:38,<NA>
